# Run AI Lung Segmentation
## JA Engelbrecht

#### Import Libraries and Functions

In [ ]:
# ===========================Don't Change===============================
import UNets.MyImplementation.UNET_Adv7 as UNet
from tensorflow.keras.optimizers import Adam

from MyFunctions.CreatePaths import CreatePaths
from MyFunctions.LoadImages import LoadImages
from MyFunctions.RunModels import RunModels

import SimpleITK as sitk
import tensorflow as tf
import pandas as pd
import numpy as np
import importlib
import os

from os import listdir
from os.path import isfile, join

In [ ]:
print(tf.__version__)

#### Create Helper Functions

In [ ]:
# ===========================Don't Change===============================
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def loadWeights(modelStringName):
    model = UNet.UNet_Adv(input_shape=(256, 256, 1),
                          n_filters=32, activation=tf.keras.layers.LeakyReLU(), showSummary=False).CreateUnet()
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss=dice_coef_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
    try:
        model.load_weights(modelStringName)
        return(model)
        # print("Success!")
    except:
        print("Error when loading weights..")

#### Initialise Default Variables

In [ ]:
# ===========================Don't Change===============================
MODEL_PATH = os.path.join(os.path.abspath(''), 'TrainedModels\\')
Orientations = ["Axial", "Sagittal", "Coronal"]

#### Load Models

In [ ]:
Model_Axial = loadWeights(MODEL_PATH+"U-Net_7_2_Axial.h5")
Model_Sagittal = loadWeights(MODEL_PATH+"U-Net_7_2_Sagittal.h5")
Model_Coronal = loadWeights(MODEL_PATH+"U-Net_7_2_Coronal.h5")

Models = [Model_Axial, Model_Sagittal, Model_Coronal]

#### User Vairables

In [ ]:
# ==========================User Variables==============================
# Folder Location of  Images -> Change this!
Scan_Path = 'C:\\Users\\malherbe\\OneDrive - Stellenbosch University\\Jandre Masters\\CB_named_scans\\CB_100'
# Scan Name -> Change this!
Scan_Name = 'CB_100_CT_m1.nii'
# Desired Output Location -> Change this!
Output_Path = 'C:\\Users\\malherbe\\OneDrive - Stellenbosch University\\Jandre Masters\\CB_named_scans\\CB_100'

#### Run Models

In [ ]:
Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=Scan_Name,
                                      ImgPath=Scan_Path, Orientations=Orientations).LoadScanEnsamble()
RunModels(OutPath=Output_Path, ScanName="E"+Scan_Name, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
          Scan_Metadata=MetaData, Model=Models, Orientation="", Orientations=Orientations, Threshold=0.5, Batch_Size=5).runModels(Limit=True)

#### Run the following to get all three predictions separately

In [ ]:
for i in range(0,3):
    TestImage, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=Scan_Name,
                                                ImgPath=Scan_Path, Orientation=Orientations[i]).LoadScan()
    RunModels(OutPath=Output_Path, ScanName="P"+Scan_Name, Scan=TestImage,Scans="None", Scan_Size=Orig_Size,
                  Scan_Metadata=MetaData, Model=Models[i], Orientation=Orientations[i]).runModel()